# Guia Prático de Ciência de Dados com MLflow


## Conteúdos Abordados
1. **MLflow**
2. **Extração de Dados**
3. **Tipos de Dados**
4. **Engenharia de Features**
5. **Armazenamento e Versionamento de Features com MLflow**
    

## Importando Bibliotecas Necessárias

In [ ]:

# Manipulação e visualização de dados
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Bibliotecas para aprendizado de máquina
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# MLflow para rastreamento de experimentos
import mlflow

# Supressão de avisos
import warnings
warnings.filterwarnings("ignore")
    

## Noções Básicas sobre o MLflow

In [ ]:

# Iniciar um experimento no MLflow
mlflow.start_run()

# Registrar um parâmetro
mlflow.log_param("parametro1", "valor1")

# Registrar uma métrica
mlflow.log_metric("metrica1", 0.85)

# Encerrar o experimento
mlflow.end_run()

print("Experimento do MLflow registrado com sucesso.")
    

## Extração de Dados

In [ ]:

# Exemplo: Importando um dataset do Kaggle
# Antes de rodar, certifique-se que a API do Kaggle está configurada e kaggle.json está em ~/.kaggle
# Instalar Kaggle caso não esteja instalado: !pip install kaggle

# Comando para baixar um dataset do Kaggle
# Exemplo: Dataset do Titanic
!kaggle competitions download -c titanic

# Descompactar o arquivo
!unzip titanic.zip -y

# Carregar o dataset
dados = pd.read_csv("train.csv")
dados.head()
    

## Entendimento e Interpretação dos Dados

In [ ]:

# Exibir informações sobre o dataset
print(dados.info())

# Exibir estatísticas básicas
print(dados.describe())

# Verificar valores ausentes
print(dados.isnull().sum())
    

## Limpeza dos Dados

In [ ]:
# Tratamento de valores ausentes apenas em colunas numéricas (mediana)
numeric_cols = dados.select_dtypes(include=[np.number]).columns
medianas = dados[numeric_cols].median()
dados[numeric_cols] = dados[numeric_cols].fillna(medianas)

# Removendo duplicatas
dados.drop_duplicates(inplace=True)

print("Dados limpos com sucesso!")

## Trabalhando com Dados Categóricos

In [ ]:

# Exemplo: One-hot encoding para features categóricas
encoder = OneHotEncoder(sparse=False)
dados_categoricos = pd.DataFrame(
    encoder.fit_transform(dados[['Sex']]),
    columns=encoder.get_feature_names_out(['Sex'])
)

# Adicionar os dados codificados de volta ao dataset principal
dados = pd.concat([dados, dados_categoricos], axis=1)
dados.drop('Sex', axis=1, inplace=True)
dados.head()
    

## Trabalhando com Dados Quantitativos

In [ ]:

# Exemplo: Padronização de features numéricas
scaler = StandardScaler()
dados_numericos = pd.DataFrame(
    scaler.fit_transform(dados[['Age', 'Fare']]),
    columns=['Age_escalado', 'Fare_escalado']
)

# Adicionar os dados escalonados de volta ao dataset principal
dados = pd.concat([dados, dados_numericos], axis=1)
dados.drop(['Age', 'Fare'], axis=1, inplace=True)
dados.head()
    

## Armazenamento e Versionamento de Features com MLflow

In [ ]:

# Salvar o dataset processado como um arquivo CSV
processed_data_path = "dados_processados.csv"
dados.to_csv(processed_data_path, index=False)
print("Dataset processado salvo localmente.")

# Registrar o dataset processado como artefato no MLflow
mlflow.start_run()  # Iniciar um novo experimento
mlflow.log_artifact(processed_data_path)  # Registrar o arquivo como artefato
mlflow.end_run()  # Encerrar o experimento

print("Features armazenadas e versionadas com sucesso no MLflow!")
    